# Philosophy of Testing

Testing is the cornerstone of *scalable* software engineering.  If you *ever* plan to contribute to an open source project (*strongly* encouraged if you wanna work in tech), learn to love your tests.  Generally speaking, they come in two categories: **unit tests** and **integration tests**.

**Unit tests** compare output of a single function to an expected result, and are *your contract with developers*.  Whenever you write or modify a function, it is *good practice* to provide test cases.  This is *especially* important when your method depends on helper functions which may get updated over time.

**Integration tests** compare the output of an entire application/algorithm, and are *your contract with customers*.  An integration test could make sure a web-page is served and rendered correctly, or it could check that a numerical PDE-solver returns a final solution within allowed tolerances.

As a codebase becomes more complex, the distinction between unit-tests and integration-tests can be fuzzy, and the "contracts" begin to break down (development slows down and user-facing bugs start appearing).  For this reason, it is *good practice* to write code with testing in mind.  Where possible, functions should be [pure](https://en.wikipedia.org/wiki/Pure_function) and [modular](https://en.wikipedia.org/wiki/Modular_programming).

# Instructions

In this notebook, we have provided **unit tests** for your `classical_gram_schmidt` and `modified_gram_schmidt` implementations.  We have *also* provided a working **implementation** of CGS for reference - ultimately, you should replace our implementation with your own.

First, execute the following cell (`<Shift+Enter>`) to run some imports.  Note that your implementation should only depend on `import numpy as np` - the other imports are used for testing.

In [14]:
%pip install parameterized
%pip install ipython_unittest
%reload_ext ipython_unittest

import numpy as np
import sympy as sp
import unittest
from parameterized import parameterized


Next, paste in your implementations of CGS and MGS.  Each function should take a matrix and return a pair of matrices (Q, R).

In [4]:
import numpy as np

def classical_gram_schmidt(A: np.ndarray):
    """Returns the QR decomposition of A using the classical gram-schmidt algorithm.

    Arguments:
        A (np.ndarray): A matrix whose columns are linearly independent.
    Returns:
        (Q, R): The QR decomposition of A.
    """
    A = np.asanyarray(A, dtype=np.float64)
    m, n = A.shape
    assert m >= n

    Q = np.zeros_like(A)
    R = np.zeros((n, n), dtype=A.dtype)
    for j in range(n):
        Q[:, j] = A[:, j]
        for k in range(j):
            R[k, j] = Q[:, k] @ A[:, j]
            Q[:, j] -= R[k, j] * Q[:, k]
        R[j, j] = np.linalg.norm(Q[:, j], ord=2)
        Q[:, j] /= R[j, j]
    return Q, R

def modified_gram_schmidt(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Returns the QR decomposition of A using the modified gram-schmidt algorithm.

    Arguments:
        A (np.ndarray): A matrix whose columns are linearly independent.
    Returns:
        (Q, R): The QR decomposition of A.
    """
    # Paste your implementation here!

This final cell contains the promised **unit tests**.  We're using [sympy](https://www.sympy.org/) to compute *exact* QR-decompositions (symbolically), and evaluating these as `float64`.  Sympy and other [CAS](https://en.wikipedia.org/wiki/Computer_algebra_system) frameworks are *great* for solving things that you'd typically do on pen-and-paper, but they don't scale well to larger systems.

Notice that we're checking $Q$ for *relative error* (`rtol`), and $R$ for *absolute error* (`atol`).  You can read more about tolerances in the [np.allclose documentation](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html#numpy.allclose).

In [13]:
%%unittest_main

class HW01_Tests(unittest.TestCase):

    @staticmethod
    def _qr_exact(A, dtype=np.float64):
        # Rather than hard-coding matrices with 16-digits of
        # precision, use Sympy to compute an exact *symbolic*
        # solution to QR(A).
        Q, R = sp.Matrix(A).QRdecomposition()
        # Truncate the result to the specified dtype.
        return (np.array(Q, dtype), np.array(R, dtype))


    @parameterized.expand([
        # CGS solves this quite well.
        ([[1, 1], [1, 0]], ),
        # But struggles here....
        ([[1000, 1001], [1000, 999]], 3e-13, 1e-9),
        # Make sure to handle m>n!
        ([[1, 0], [0, 1], [1, 1]], ),
    ])
    def test_classical(self, A, rtol=1e-15, atol=1e-15):
        Qexpect, Rexpect = self._qr_exact(A)
        Qobserved, Robserved = classical_gram_schmidt(
            np.asarray(A, dtype=np.float64))
        
        np.testing.assert_allclose(Qexpect, Qobserved, rtol=rtol, err_msg="Mismatched Q.")
        np.testing.assert_allclose(Rexpect, Robserved, atol=atol, err_msg="Mismatched R.")

        np.testing.assert_allclose(
            np.eye(Qobserved.shape[1]), Qobserved.T @ Qobserved,
            atol=atol, err_msg="Q is not orthonormal.")

        np.testing.assert_allclose(
            np.triu(Robserved), Robserved,
            err_msg="R is not upper triangular.")


    @parameterized.expand([
        # MGS also handles this.
        ([[1, 1], [1, 0]], ),
        # And does much better here.
        ([[1000, 1001], [1000, 999]], 3e-13, 3e-13),
        # Make sure to handle m>n!
        ([[1, 0], [0, 1], [1, 1]], ),
    ])
    def test_modified(self, A, rtol=1e-15, atol=1e-15):
        Qexpect, Rexpect = self._qr_exact(A)
        Qobserved, Robserved = modified_gram_schmidt(
            np.asarray(A, dtype=np.float64))
        
        np.testing.assert_allclose(Qexpect, Qobserved, rtol=rtol, err_msg="Mismatched Q.")
        np.testing.assert_allclose(Rexpect, Robserved, atol=atol, err_msg="Mismatched R.")

        np.testing.assert_allclose(
            np.eye(Qobserved.shape[1]), Qobserved.T @ Qobserved,
            atol=atol, err_msg="Q is not orthonormal.")

        np.testing.assert_allclose(
            np.triu(Robserved), Robserved,
            err_msg="R is not upper triangular.")
